# Librerias

In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset
from google.colab import files

In [ ]:


print("\n" + "="*60)
print("🏆 PREDICCIONES PARA COMPETENCIA")
print("="*60)

# Definir la ruta donde se encuentran los archivos del modelo y tokenizador
model_path = "/ruta/a/tu/carpeta/modelo"
tokenizer_path = "/ruta/a/tu/carpeta/tokenizer"

# Cargar el tokenizador y el modelo desde las carpetas locales
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Mover el modelo a un dispositivo (GPU si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Cargar dataset de competencia
dataset_path = "/content/TA1C_dataset_detection_test.csv"

if not os.path.exists(dataset_path):
    print(f"❌ Error: No se encuentra {dataset_path}")
    print("📥 Sube el archivo de competencia a Colab")
    print("   El archivo debe tener columnas: 'Tweet ID', 'Teaser Text'")
else:
    print("📂 Cargando dataset de competencia...")
    dat_competencia = pd.read_csv(dataset_path)
    print(f"✅ Dataset cargado: {len(dat_competencia)} ejemplos")
    print(f"📋 Columnas: {list(dat_competencia.columns)}")

    # Función para reemplazar URLs (mismo preprocesamiento que entrenamiento)
    def reemplazar_urls(df, columna_texto):
        """Reemplazar URLs por token [URL]"""
        patron_url = r'https?://\S+|www\.\S+'
        df = df.copy()
        df[columna_texto] = df[columna_texto].apply(
            lambda texto: re.sub(patron_url, '[URL]', str(texto))
        )
        return df

    # Aplicar preprocesamiento
    print("🔄 Aplicando preprocesamiento...")
    dat_test = reemplazar_urls(dat_competencia, "Teaser Text")
    dat_test["Teaser Text"] = dat_test["Teaser Text"].apply(limpiar_texto)

    print("📋 Ejemplos de preprocesamiento:")
    for i in range(min(3, len(dat_test))):
        original = dat_competencia.iloc[i]["Teaser Text"]
        procesado = dat_test.iloc[i]["Teaser Text"]
        print(f"   Original: {original[:60]}...")
        print(f"   Procesado: {procesado[:60]}...")
        print()

    # Función de predicción optimizada para competencia
    def predecir_competencia(textos, batch_size=8):
        """Predecir textos para competencia por lotes"""
        model.eval()
        todas_predicciones = []

        print(f"🔄 Procesando {len(textos)} textos en lotes de {batch_size}...")

        for i in tqdm(range(0, len(textos), batch_size), desc="Prediciendo para competencia"):
            batch_textos = textos[i:i+batch_size]

            try:
                # Tokenizar lote
                inputs = tokenizer(
                    batch_textos,
                    return_tensors="pt",
                    truncation=True,
                    padding=True,
                    max_length=512
                )

                # Mover a dispositivo
                inputs = {k: v.to(device) for k, v in inputs.items()}

                # Predicción
                with torch.no_grad():
                    outputs = model(**inputs)
                    pred_classes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

                todas_predicciones.extend(pred_classes)

            except Exception as e:
                print(f"⚠️ Error en lote {i//batch_size}: {e}")
                # En caso de error, predecir "No" para el lote
                todas_predicciones.extend([0] * len(batch_textos))

        return todas_predicciones

    # Realizar predicciones para competencia
    print("🚀 Realizando predicciones para competencia...")
    textos = dat_test["Teaser Text"].tolist()
    tweet_ids = dat_test["Tweet ID"].tolist()

    predicciones_numericas = predecir_competencia(textos, batch_size=8)
    predicciones_texto = ["Clickbait" if pred == 1 else "No" for pred in predicciones_numericas]

    print("✅ Predicciones completadas!")

    # Crear DataFrame de resultados
    df_resultados = pd.DataFrame({
        "Tweet ID": tweet_ids,
        "Tag Value": predicciones_texto
    })

    # Verificaciones de calidad
    print(f"\n🔍 VERIFICACIONES DE CALIDAD:")
    print(f"   Total predicciones: {len(df_resultados)}")
    print(f"   Distribución:")
    distribucion = df_resultados['Tag Value'].value_counts()
    for valor, cantidad in distribucion.items():
        porcentaje = (cantidad / len(df_resultados)) * 100
        print(f"     {valor}: {cantidad} ({porcentaje:.1f}%)")

    # Verificar formato
    print(f"   Valores únicos: {df_resultados['Tag Value'].unique()}")
    print(f"   Valores nulos: {df_resultados.isnull().sum().sum()}")

    if df_resultados.isnull().sum().sum() == 0:
        print("   ✅ Sin valores nulos")
    else:
        print("   ⚠️ Hay valores nulos")

    if set(df_resultados['Tag Value'].unique()) == {"No", "Clickbait"}:
        print("   ✅ Formato correcto")
    else:
        print("   ⚠️ Formato incorrecto")

    # Guardar archivo
    ruta_salida = "/content/detection2.csv"
    df_resultados.to_csv(ruta_salida, index=False)

    print(f"\n💾 Archivo guardado: {ruta_salida}")

    # Verificar archivo guardado
    if os.path.exists(ruta_salida):
        size_mb = os.path.getsize(ruta_salida) / (1024 * 1024)
        print(f"   Tamaño: {size_mb:.2f} MB")

        # Mostrar muestra del archivo
        print(f"\n📋 Muestra del archivo generado:")
        print(df_resultados.head(10))

    # Descargar archivo
    print(f"\n📥 Descargando archivo...")
    try:
        files.download(ruta_salida)
        print("✅ ¡Archivo descargado exitosamente!")
    except Exception as e:
        print(f"⚠️ Error al descargar: {e}")
        print(f"El archivo está guardado en: {ruta_salida}")

    # Resumen final
    total_clickbait = (df_resultados['Tag Value'] == 'Clickbait').sum()
    porcentaje_clickbait = (total_clickbait / len(df_resultados)) * 100

    print(f"\n🎯 RESUMEN FINAL:")
    print("="*50)
    print(f"🏆 Modelo usado: Llama")
    print(f"📁 Archivo: detection_checkpoint2760.csv")
    print(f"📊 Total predicciones: {len(df_resultados)}")
    print(f"🎯 Clickbait detectado: {total_clickbait} ({porcentaje_clickbait:.1f}%)")
    print(f"✅ Mejor modelo (menos sobreajuste)")
    print(f"📈 Métricas en test: Acc={accuracy:.3f}, F1={f1:.3f}")
    print("="*50)
    print("🏆 ¡Listo para enviar a la competencia!")
